In [0]:
%sql
CREATE TABLE IF NOT EXISTS odace.map.silver_lignes (
  code_ligne STRING COMMENT 'Code de la ligne',
  lib_ligne STRING COMMENT 'Libellé de la ligne',
  catlig STRING COMMENT 'Catégorie de la ligne',
  isTGV STRING COMMENT '1 if it is TGV',
  rg_troncon INT COMMENT 'Rang du tronçon',
  pkd STRING COMMENT 'Point kilométrique de début',
  pkf STRING COMMENT 'Point kilométrique de fin',
  idgaia STRING COMMENT 'Identifiant GAIA',
  x_d_l93 DOUBLE COMMENT 'Coordonnée X début (Lambert 93)',
  y_d_l93 DOUBLE COMMENT 'Coordonnée Y début (Lambert 93)',
  x_f_l93 DOUBLE COMMENT 'Coordonnée X fin (Lambert 93)',
  y_f_l93 DOUBLE COMMENT 'Coordonnée Y fin (Lambert 93)',
  x_d_wgs84 DOUBLE COMMENT 'Coordonnée X début (WGS84)',
  y_d_wgs84 DOUBLE COMMENT 'Coordonnée Y début (WGS84)',
  x_f_wgs84 DOUBLE COMMENT 'Coordonnée X fin (WGS84)',
  y_f_wgs84 DOUBLE COMMENT 'Coordonnée Y fin (WGS84)',
  c_geo_d STRING COMMENT 'Code géographique début',
  c_geo_f STRING COMMENT 'Code géographique fin',
  geo_point STRING COMMENT 'Point géographique',
  geo_shape STRING COMMENT 'Forme géographique',
  ingestion_timestamp TIMESTAMP COMMENT 'Horodatage d\'ingestion'
)
USING DELTA;

In [0]:
%sql
MERGE INTO odace.map.silver_lignes AS silver
USING (
  SELECT
    code_ligne,
    lib_ligne,
    catlig,
    CASE WHEN catlig = 'Ligne à grande vitesse' THEN '1' ELSE '0' END AS isTGV,
    CAST(rg_troncon AS INT) AS rg_troncon,
    pkd,
    pkf,
    idgaia,
    x_d_l93,
    y_d_l93,
    x_f_l93,
    y_f_l93,
    x_d_wgs84,
    y_d_wgs84,
    x_f_wgs84,
    y_f_wgs84,
    c_geo_d,
    c_geo_f,
    geo_point,
    geo_shape,
    ingestion_timestamp
  FROM (
    SELECT *,
      ROW_NUMBER() OVER (PARTITION BY code_ligne ORDER BY ingestion_timestamp DESC) AS rn
    FROM odace.map.bronze_lignes
  ) t
  WHERE rn = 1
) AS bronze
ON silver.code_ligne = bronze.code_ligne
WHEN MATCHED THEN UPDATE SET
  silver.lib_ligne = bronze.lib_ligne,
  silver.catlig = bronze.catlig,
  silver.isTGV = bronze.isTGV,
  silver.rg_troncon = bronze.rg_troncon,
  silver.pkd = bronze.pkd,
  silver.pkf = bronze.pkf,
  silver.idgaia = bronze.idgaia,
  silver.x_d_l93 = bronze.x_d_l93,
  silver.y_d_l93 = bronze.y_d_l93,
  silver.x_f_l93 = bronze.x_f_l93,
  silver.y_f_l93 = bronze.y_f_l93,
  silver.x_d_wgs84 = bronze.x_d_wgs84,
  silver.y_d_wgs84 = bronze.y_d_wgs84,
  silver.x_f_wgs84 = bronze.x_f_wgs84,
  silver.y_f_wgs84 = bronze.y_f_wgs84,
  silver.c_geo_d = bronze.c_geo_d,
  silver.c_geo_f = bronze.c_geo_f,
  silver.geo_point = bronze.geo_point,
  silver.geo_shape = bronze.geo_shape,
  silver.ingestion_timestamp = bronze.ingestion_timestamp
WHEN NOT MATCHED THEN INSERT (
  code_ligne, lib_ligne, catlig, isTGV, rg_troncon, pkd, pkf, idgaia, x_d_l93, y_d_l93, x_f_l93, y_f_l93, x_d_wgs84, y_d_wgs84, x_f_wgs84, y_f_wgs84, c_geo_d, c_geo_f, geo_point, geo_shape, ingestion_timestamp
) VALUES (
  bronze.code_ligne, bronze.lib_ligne, bronze.catlig, bronze.isTGV, bronze.rg_troncon, bronze.pkd, bronze.pkf, bronze.idgaia, bronze.x_d_l93, bronze.y_d_l93, bronze.x_f_l93, bronze.y_f_l93, bronze.x_d_wgs84, bronze.y_d_wgs84, bronze.x_f_wgs84, bronze.y_f_wgs84, bronze.c_geo_d, bronze.c_geo_f, bronze.geo_point, bronze.geo_shape, bronze.ingestion_timestamp
);